In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configuración de los generadores de imágenes
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'ruta/a/tu/carpeta',  # Carpeta raíz que contiene las subcarpetas "malignas" y "benignas"
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',  # Clasificación binaria
    subset='training'  # Parte del conjunto de datos para entrenamiento
)

validation_generator = train_datagen.flow_from_directory(
    'ruta/a/tu/carpeta',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Parte del conjunto de datos para validación
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Creación del modelo
model = Sequential()

# Capas convolucionales
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Aplanamiento de la capa convolucional para la entrada a las capas densas
model.add(Flatten())

# Capas densas
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Salida con una neurona (maligno o benigno)

# Compilación del modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Entrenamiento del modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # Puedes ajustar el número de épocas
)


In [ ]:
# Evaluación del modelo
loss, accuracy = model.evaluate(validation_generator)
print(f'Pérdida en validación: {loss}')
print(f'Precisión en validación: {accuracy}')


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array)
    if prediction > 0.5:
        print("La lesión es maligna")
    else:
        print("La lesión es benigna")
